In [1]:
import pandas as pd
import polars as pl
from data import *
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from missforest import MissForest
import pickle
from lightgbm import LGBMClassifier, LGBMRegressor
import pyarrow
import sklearn
from sklearn.svm import SVR, LinearSVR
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os
import numpy as np
import shap
from scipy import stats
from typing import Any, cast, Final
from utils import *
import wrds
from xgboost import XGBRegressor

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
full_data = pl.read_csv(get_data_file_path("1_4_complete_dataset.csv"), dtypes=ALL_DATA_WITH_DATES_ENTRY_SCHEMA).to_pandas()

In [5]:
import pandas as pd

def winsorize_columns(df: pd.DataFrame, quantile: float = 0.01) -> pd.DataFrame:
  """
  Winsorizes each column (from the 4th onwards) independently in a DataFrame.

  Args:
      df: The input DataFrame.
      quantile: The winsorization quantile (default: 0.01 for 1%).

  Returns:
      A new DataFrame with winsorized columns.
  """
  winsorized_df = df.copy()
  for col in df.columns[3:]:
    lower_bound = df[col].quantile(quantile)
    upper_bound = df[col].quantile(1 - quantile)
    winsorized_df[col] = winsorized_df[col].clip(lower=lower_bound, upper=upper_bound)
  return winsorized_df

# Assuming 'full_data' is your already sorted DataFrame
winsorized_data = winsorize_columns(full_data.copy(), quantile=0.01)  # Winsorize on a copy

# Now you can use 'winsorized_data' for further processing


In [6]:
winsorized_data.describe()

,permno,date,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,...,std_turn,tang,zerotrade,zerotradeAlt1,zerotradeAlt12,STreversal,Price,Size,date_right,ret
count,5.001719e+06,5001719,2.934379e+06,1.157850e+06,2.315298e+06,2.958277e+06,213321.000000,108731.000000,30170.000000,1.005624e+06,...,2.064645e+06,1.398719e+06,4.288348e+06,4.453415e+06,4.035377e+06,5.001692e+06,4.889704e+06,4.889704e+06,5001719,4.918560e+06
mean,5.255790e+04,1994-01-21 01:42:15.297000,3.027881e+00,-1.802403e+00,-3.428303e-03,2.875257e-02,0.483201,14.891954,9.242051,6.257750e-02,...,-1.125646e-01,6.912801e-01,1.469744e+00,1.554276e+00,1.403133e+00,-7.159066e-01,-2.495770e+00,-1.156022e+01,1994-03-01 01:15:24.042000,7.377432e-03
min,1.000000e+04,1925-12-01 00:00:00,7.494592e-02,-4.093309e+01,-3.792033e-01,-3.033897e-01,0.000000,9.000000,0.000000,1.677615e-04,...,-2.216114e+00,2.157037e-01,2.167118e-09,1.435528e-09,2.456003e-09,-5.335041e+01,-4.971894e+00,-1.730189e+01,1932-04-30 00:00:00,-3.829790e-01
25%,2.096900e+04,1982-01-01 00:00:00,6.433154e-01,-1.355855e+00,-5.248681e-02,-1.260950e-02,0.000000,13.000000,0.000000,4.987868e-03,...,-8.703909e-02,6.002009e-01,2.026757e-08,1.724399e-08,2.139086e-08,-5.984600e+00,-3.354280e+00,-1.307093e+01,1982-01-29 00:00:00,-5.782625e-02
50%,5.489400e+04,1996-10-01 00:00:00,1.384571e+00,-3.936195e-01,-7.283284e-03,2.834095e-02,0.000000,15.000000,7.438817,1.651571e-02,...,-3.480813e-02,6.968244e-01,6.731287e-08,5.016651e-08,7.898551e-08,0.000000e+00,-2.682732e+00,-1.142684e+01,1996-10-31 00:00:00,0.000000e+00
75%,8.071700e+04,2010-05-01 00:00:00,3.108830e+00,-6.500524e-02,3.915805e-02,7.099786e-02,1.000000,17.000000,10.728353,5.642348e-02,...,-1.556216e-02,7.957041e-01,8.590912e-01,9.545455e-01,9.048957e-01,5.555600e+00,-1.802122e+00,-9.910483e+00,2010-05-28 00:00:00,6.211200e-02
max,9.343600e+04,2023-12-01 00:00:00,3.243325e+01,1.000000e+00,4.429111e-01,3.799396e-01,1.000000,21.000000,80.000000,8.886826e-01,...,-2.255419e-03,1.108704e+00,1.540347e+01,1.718182e+01,1.457526e+01,3.818180e+01,1.268511e+00,-6.894290e+00,2023-07-31 00:00:00,5.396830e-01
std,2.950973e+04,NaN,4.889376e+00,5.294965e+00,1.141838e-01,9.759911e-02,0.499719,2.690423,12.468353,1.316377e-01,...,2.854031e-01,1.709446e-01,3.244883e+00,3.626017e+00,3.062318e+00,1.361686e+01,1.246024e+00,2.252961e+00,NaN,1.380761e-01


In [7]:
winsorized_data

,permno,yyyymm,date,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,...,std_turn,tang,zerotrade,zerotradeAlt1,zerotradeAlt12,STreversal,Price,Size,date_right,ret
0,10000,198601,1986-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0000,-1.475907,-9.686575,1986-01-31,0.539683
1,10000,198602,1986-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.785175e-08,NaN,25.7143,-1.178655,-9.389323,1986-02-28,-0.257143
2,10000,198603,1986-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.023392e-07,NaN,-36.5385,-1.490091,-9.700759,1986-03-31,0.365385
3,10000,198604,1986-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,7.467463e-08,NaN,9.8592,-1.386294,-9.627207,1986-04-30,-0.098592
4,10000,198605,1986-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,7.649551e-08,NaN,22.2656,-1.134423,-9.375336,1986-05-30,-0.222656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5001714,93436,202308,2023-08-01,NaN,NaN,-0.003877,0.016862,NaN,NaN,NaN,...,NaN,0.698178,NaN,NaN,NaN,3.4962,-4.971894,-17.301890,2023-07-31,-0.034962
5001715,93436,202309,2023-09-01,NaN,NaN,-0.003877,0.016862,NaN,NaN,NaN,...,NaN,0.698178,NaN,NaN,NaN,3.0456,-4.971894,-17.301890,2023-07-31,-0.030456
5001716,93436,202310,2023-10-01,NaN,NaN,-0.003877,0.016862,NaN,NaN,NaN,...,NaN,0.698178,NaN,NaN,NaN,19.7346,-4.971894,-17.301890,2023-07-31,-0.197346
5001717,93436,202311,2023-11-01,NaN,NaN,-0.003877,0.016862,NaN,NaN,NaN,...,NaN,0.698178,NaN,NaN,NaN,-19.5379,-4.971894,-17.301890,2023-07-31,0.195379
